# Generating Light Curves from Strongly Lensed Systems in Twinkles data

<br>Owner(s): **Bryce Kalmbach** ([@jbkalmbach](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@jbkalmbach))
<br>Last Verified to Run: **2019-08-09**
<br>Verified Stack Release: **v18.0, w_2019_31**

This notebook shows how to use the truth information for the Twinkles dataset to find simulated strongly lensed AGN systems. It will then demonstrate how to match these to `src` catalogs from the butler. Using the matched catalogs across a series of visits we will then create light curves.

### Learning Objectives:

After working through this tutorial you should be able to: 
1. Find simulated strongly lensed systems in Twinkles data;
2. Match the truth catalogs to the `src` catalogs;
3. Generate light curves for observations from a sequence of visits.

This notebook uses methods from [A Guided Tour of LSST Calexps](../Basics/Calexp_guided_tour.ipynb) to load calexps and display postage stamps.

### Logistics
This notebook is intended to be runnable on `lsst-lsp-stable.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.

## Set-up

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

import lsst.afw.display as afw_display
from lsst.daf.persistence import Butler

from lsst.geom import SpherePoint, Angle, degrees
import lsst.geom as geom

from astropy.coordinates import SkyCoord
from astropy import units as u

%matplotlib inline

## Find Twinkles Lensed AGN

We have saved the Twinkles truth locations for lensing galaxies and the AGN images.

In [ ]:
def get_twinkles_truth():

    """
    Get twinkles object locations.
    
    Returns
    -------
    lens_truth: pandas dataframe
        The truth locations and additional information of the lensing galaxies for the Twinkles systems.
        
    
    agn_truth: pandas dataframe
        The truth locations and additional information for the AGN images in the Twinkles systems.
    """

    lens_truth = pd.read_csv('/project/shared/data/Twinkles_subset/truth/sprinkled_lens_230_J2000.txt')
    agn_truth = pd.read_csv('/project/shared/data/Twinkles_subset/truth/sprinkled_agn_230_J2000.txt')

    return lens_truth, agn_truth

In [ ]:
twinkles_lens_truth, twinkles_agn_truth = get_twinkles_truth()

### Visualize one of these systems in the Twinkles data

Sanity check our truth catalog by visualizing one of the systems. To do this we will use the `butler` to load a `calexp` image.

In [ ]:
# Set up a butler
datadir = '/project/shared/data/Twinkles_subset/output_data_v2/'
butler = Butler(datadir)

In [ ]:
# Get the data for one visit
calexp_subset = butler.subset('calexp')
dataId = calexp_subset.cache[0]
print(dataId)

In [ ]:
# Get the calexp
calexp = butler.get('calexp', **dataId)

Now we can use `afw_display` to load the calexp image and display it using a `matplotlib` backend.

In [ ]:
fig = plt.figure(figsize=(8,8))
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale('asinh', 'zscale')
display1.mtv(calexp.image)
# Since we are using matplotlib as a backend we can add labels using pyplot commands
plt.xlabel('X Pixels', size=28)
plt.ylabel('Y Pixels', size=28)
plt.xticks(size=18)
plt.yticks(size=18)

We've now loaded an image but need to translate the ra, dec coordinates of the truth catalog into pixel values in the image. To do this we will get the WCS information for the `calexp`.

In [ ]:
wcs = calexp.getWcs()

Now we will pick a twinkles system to look at. We will use twinkles system 27 since it has a good separation between its images.

In [ ]:
twinkles_sys = 27
test_images = twinkles_agn_truth.query('twinkles_system == %i' % twinkles_sys) 
test_images

In [ ]:
test_lens = twinkles_lens_truth.query('galid == %i' % test_images['lens_galids'].iloc[0])
test_lens

Now we use the WCS for the `calexp` and the ra, dec for our lensing galaxy and images to get pixel coordinates.

In [ ]:
x_lens, y_lens = wcs.skyToPixel(SpherePoint(Angle(float(test_lens['ra']), degrees),
                                            Angle(float(test_lens['dec']), degrees)))
print(x_lens, y_lens)

In [ ]:
x_image_1, y_image_1 = wcs.skyToPixel(SpherePoint(Angle(float(test_images['ra'].iloc[0]), degrees),
                                                  Angle(float(test_images['dec'].iloc[0]), degrees)))
x_image_2, y_image_2 = wcs.skyToPixel(SpherePoint(Angle(float(test_images['ra'].iloc[1]), degrees),
                                                  Angle(float(test_images['dec'].iloc[1]), degrees)))
print(x_image_1, y_image_1)
print(x_image_2, y_image_2)

### Display a Twinkles System

Let's set up a bounding box to create a cutout of just this system.

In [ ]:
bbox = geom.Box2I(geom.Point2I(x_lens-75, y_lens-75), geom.Extent2I(200, 150))
cutout = calexp[bbox].image

Here we display the system using our cutout.

In [ ]:
fig = plt.figure(figsize=(12,8))
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale('asinh', 'zscale')
display1.mtv(cutout)
plt.scatter(x_lens, y_lens, c='r', marker='+', label='Lens', s=400, lw=2)
plt.scatter(x_image_1, y_image_1, c='b', marker='+', label='Lensed AGN Image 1', s=400, lw=2)
plt.scatter(x_image_2, y_image_2, c='g', marker='+', label='Lensed AGN Image 2', s=400, lw=2)
plt.legend(fontsize=32)
plt.xlabel('X Pixels', size=28)
plt.ylabel('Y Pixels', size=28)
plt.xticks(size=18)
plt.yticks(size=18)

## Get Source Catalogs and match to Truth

Here we'll use the butler to get source catalogs for all the visits in the `r` band and match these source catalogs to our truth. First we use the butler to get the subset of r-band visits.

In [ ]:
r_band_subset = butler.subset('src', filter='r')

Let's take the source catalog from the first visit and make the same plot as before but this time include all identified sources in the DM catalog.

In [ ]:
src_cat = butler.get('src', dataId = r_band_subset.cache[0]).asAstropy().to_pandas()

In [ ]:
src_pix = [wcs.skyToPixel(SpherePoint(Angle(float(src_cat['coord_ra'][idx])),
                                      Angle(float(src_cat['coord_dec'][idx]))))for idx in range(len(src_cat))]

In [ ]:
fig = plt.figure(figsize=(12,8))
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale('asinh', 'zscale')
display1.mtv(cutout)
plt.scatter(x_lens, y_lens, c='r', marker='+', label='Lens', s=400, lw=2, zorder=10)
plt.scatter(x_image_1, y_image_1, c='b', marker='+', label='Lensed AGN Image 1', s=400, lw=2, zorder=10)
plt.scatter(x_image_2, y_image_2, c='g', marker='+', label='Lensed AGN Image 2', s=400, lw=2, zorder=10)

# Mark Sources
for idx in range(len(src_pix)):
    src_loc = src_pix[idx]
    display1.dot('o', src_loc[0], src_loc[1], size=4, ctype='orange')

plt.legend(fontsize=32)
plt.xlabel('X Pixels', size=28)
plt.ylabel('Y Pixels', size=28)
plt.xticks(size=18)
plt.yticks(size=18)

We see that the AGN image #2 is identified as its own source very close to the true position but that the lensing galaxy and AGN image #1 are blended together and identified as a single source in the output catalog. Let's make this easy on ourselves and focus on building a lightcurve for AGN image #2 below which should be well identified in all 10 r-band images available.

To do catalog matching we will use Astropy's `match_to_catalog_sky` method. This will require creating SkyCoord objects with the ra, dec coordinates of our truth catalog in the next cell and then in each visit doing the same with the source catalogs.

In [ ]:
lens_coords = SkyCoord(test_lens['ra']*u.deg, test_lens['dec']*u.deg)
image_coords = SkyCoord(test_images['ra']*u.deg, test_images['dec']*u.deg)

Now we will loop through all the r-band visits and get three things from the butler:

1) Source catalogs: These will have all the sources in each visit. We will use this to match to our truth catalog and get fluxes.

2) Calexps: The processed images. We will need this to get the information necessary to convert the source catalog fluxes into magntiudes with a `photoCalib`. When we have a `photoCalib` object we can use `instFluxToMagnitude` to convert `base_PsfFlux_instFlux` values in the source catalog to magnitudes.

3) Calexp Metadata: The metadata will contain information from the FITS headers of the images and we will use this to get the MJD of each visit for our light curve.

Once we have all this information for each visit we use `match_to_catalog_sky` with our truth locations and identify the source catalog entry that is our AGN image.

In [ ]:
image_mags = []
image_errs_high = []
image_errs_low = []
image_times = []

for dataId in r_band_subset.cache:
    
    # Get data products from Butler
    src_cat = butler.get('src', dataId = dataId).asAstropy().to_pandas()
    photoCalib = butler.get('calexp_photoCalib', dataId = dataId)
    metadata = butler.get('calexp_md', dataId=dataId)
    
    # Get image times from metadata
    image_times.append(metadata['MJD-OBS'])
    
    # Calculate magnitudes
    src_mags = [photoCalib.instFluxToMagnitude(flux) for flux in src_cat['base_PsfFlux_instFlux']]
    src_errs_high = [photoCalib.instFluxToMagnitude(flux+flux_err) \
                     for flux, flux_err in zip(src_cat['base_PsfFlux_instFlux'],
                                               src_cat['base_PsfFlux_instFluxErr'])]
    src_errs_low = [photoCalib.instFluxToMagnitude(flux-flux_err) \
                    for flux, flux_err in zip(src_cat['base_PsfFlux_instFlux'],
                                              src_cat['base_PsfFlux_instFluxErr'])]
    src_cat['r_mag'] = src_mags
    src_cat['r_err_high'] = src_errs_high
    src_cat['r_err_low'] = src_errs_low
    
    # Use Astropy to match catalogs and identify source that matches to AGN image we want
    visit_coords = SkyCoord(src_cat['coord_ra']*u.rad, src_cat['coord_dec']*u.rad)
    idx, sep2d, sep3d = image_coords.match_to_catalog_sky(visit_coords)
    
    # Use index 1 since we want the second of our two images. Save magnitude information for our light curve.
    image_mags.append(src_cat.iloc[idx[1]]['r_mag'])
    image_errs_high.append(src_cat.iloc[idx[1]]['r_err_high'] - src_cat.iloc[idx[1]]['r_mag'])
    image_errs_low.append(src_cat.iloc[idx[1]]['r_mag'] - src_cat.iloc[idx[1]]['r_err_low'])

Now we have all the information we need to plot the light curve below!

In [ ]:
fig = plt.figure(figsize=(9, 6))
plt.errorbar(image_times, image_mags, yerr=[image_errs_high, image_errs_low], marker='o', lw=2)
plt.scatter(image_times, image_mags, c='r', zorder=10)
plt.gca().invert_yaxis()
plt.xlabel('MJD', size=18)
plt.ylabel('r-band Magnitude', size=18)
plt.title('Light Curve for AGN image #2', size=24)
plt.xticks(size=14)
plt.yticks(size=14)